In [8]:
import cv2
import numpy as np
import json

# Load calibration parameters from JSON file
with open('cameraParams.json', 'r') as file:
    params = json.load(file)

# Convert the loaded data to numpy arrays
camera_matrix = np.array(params['IntrinsicMatrix'])
radial_distortion = np.array(params['RadialDistortion'])
tangential_distortion = np.array(params['TangentialDistortion'])

# OpenCV expects at least 5 distortion coefficients: k1, k2, p1, p2, k3
# If you only have k1 and k2 (as it seems in this case), you can append zeros for p1, p2, and k3
dist_coeffs = np.hstack((radial_distortion, tangential_distortion, [0]))

# Assuming a fixed depth Z
Z = 1000  # Replace with the actual known depth value

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Optical center (principal point) from the camera matrix
    cx = int(camera_matrix[0, 2])
    cy = int(camera_matrix[1, 2])
    
    # Choose an offset from the optical center (for example, 100 pixels right and 50 pixels down)
    offset_x = -100
    offset_y = 50
    pixel_coords_offset = np.array([[[cx + offset_x, cy + offset_y]]], dtype='float32')
    
    # Convert the pixel coordinates to real-world coordinates
    undistorted_pixel_coords = cv2.undistortPoints(pixel_coords_offset, camera_matrix, dist_coeffs, P=camera_matrix)
    X = (undistorted_pixel_coords[0][0][0] * Z) / camera_matrix[0, 0]
    Y = (undistorted_pixel_coords[0][0][1] * Z) / camera_matrix[1, 1]
    
    # Overlay the optical center on the live feed
    cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
    cv2.putText(frame, "Optical Center", (cx - 50, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Overlay the offset point and its real-world coordinates
    cv2.circle(frame, (cx + offset_x, cy + offset_y), 5, (0, 0, 255), -1)
    coord_text = f"World Coords: X={X:.2f}, Y={Y:.2f}, Z={Z:.2f}"
    cv2.putText(frame, coord_text, (cx + offset_x + 10, cy + offset_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    # Show the frame
    cv2.imshow('Live Stream with Coordinates', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [12]:
import roboticstoolbox as rtb
import numpy as np

# Create a rotation matrix (-90 degrees around the Z axis)
Rz = rtb.ET.Rz(-np.pi/2).A()
Ry = rtb.ET.Ry(-np.pi/2).A()
dT = np.array([-222.42, 367.68, 484.05])

R_c = np.matmul(Rz, Ry)
R_c[:3, 3] = dT

P0 = np.ones((4, ))
P1 = np.array([-222.63, 47.3, 433.01])
P0[:3] = P1

R_c_inv = np.linalg.inv(R_c)
P2 = np.matmul(R_c_inv, P0)
P2

array([-5.1040e+01, -2.1000e-01, -3.2038e+02,  1.0000e+00])

In [5]:
R_z = rtb.ET.Rz(-90, 'degrees').A()[:3, :3]
P0 = [100, 100, -400]
np.matmul(R_z, P0)

array([ 100., -100., -400.])

In [13]:
from delta_robot import *
r = 139.6
h = 200
s = 49.84
k = 400
Zt = 0
robot = DeltaRobot(r, h, s, k, Zt)

robot.calculate_fwd_kinematics(20, 0, 0)

array([ -50.62,   -0.  , -296.89])

In [9]:
robot.calculate_inv_kinematics(49.28, 29.98, -379.06)

[-9.43, -15.39, -21.41]